Index-Building:
The factors used include the following:
- planet mass (Mjup)
- planet radius (Rjup)
- planet temperature (K)
- planet density CALCULATE
- radiation intensity (W/m^2) CALCULATE
- star distance (pc)
- AND/OR HZ measurement CALCULATE
- star metallicity ?? fraction
- star mass (Msun)
- sun radius (Rsun)
- star age ?? Gy
- star effective temperature (K)
- interactions?

The index will consist of the following:
- planet mass +
planet radius +
planet temperature +
star distance (HZ) + 
radiation intensity

Weighing can be determined in 1 of 2 ways:
- use a correlation regression as done in Cobb Douglas
- use difference from "ideal" Earth on a normal distribution to determine the importance of outliers

1. Import data and packages

In [1]:
# import packages
import numpy as np
import pandas as pd
import math 
from math import pi
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)  # None means no limit
pd.set_option('display.max_rows', None) 


# import data
data = pd.read_csv('C:/Users/21sko/Desktop/dissertation/data/exo_data.csv')

#display data
# print(data)

2. Calculate density

2.1 Convert: 
- M(jup) to M(earth)
- R(jup) to R(earth)

In [2]:
# M(jup) to M(earth)
def mjup_to_mearth(planet_mass_mjup):
    mass_mearth = planet_mass_mjup * 317.8 

    return mass_mearth

data['planet_mass_mearth'] = data.apply(lambda row: mjup_to_mearth(row['planet_mass_mjup']), axis = 1)



# R(jup) to R(earth)
def rjup_to_rearth(planet_radius_rjup):
    radius_rearth = planet_radius_rjup * 11.2

    return radius_rearth

data['planet_radius_rearth'] = data.apply(lambda row: rjup_to_rearth(row['planet_radius_rjup']), axis = 1)

# print(data)

2.2 Calculate volume 

In [3]:
# 4/3 * pi* r^3
def volume_formula(planet_radius_rearth):
    volume = (4/3) * np.pi * (planet_radius_rearth ** 3)

    return volume

data['planet_volume_rearth3'] = data.apply(lambda row: volume_formula(row['planet_radius_rearth']), axis = 1)

# print(data)

2.3 Calculate relative density

In [4]:
# D = M / V

def density_formula(planet_mass_mearth, planet_volume_rearth):
    density = (planet_mass_mearth) / (planet_volume_rearth)
    
    return density


data['planet_density_relative'] = data.apply(lambda row: density_formula(row['planet_mass_mearth'], row['planet_radius_rearth']), axis = 1)

# print(data)

In [ ]:
print(data)

2.4 Calculate absolute density = not sure if density is relevant due to a large variety of densities 

In [ ]:
# 1 D(earth) = 5.52 g/cm^3

def density_relative_to_empirical(planet_density_relative):
    planet_density_empirical = planet_density_relative * 5.52
    return planet_density_empirical

data['planet_density_empirical'] = data['planet_density_relative'].apply(density_relative_to_empirical) 

print(data)

3. Calculate Habitability Zone boundaries

3.1 Version 1: simpler

In [ ]:
# Function to convert mass to luminosity 
def mass_to_luminosity(star_mass_msun):
    return star_mass_msun  ** 3.6

# Function to calculate habitable zone boundaries 
def calculate_habitable_zone(Luminosity_Lsun):
    inner_boundary = np.sqrt(Luminosity_Lsun / 1.1)
    outer_boundary = np.sqrt(Luminosity_Lsun / 0.53)
    return inner_boundary, outer_boundary 

# Convert mass to luminosity 
data['Luminosity_Lsun'] = data['star_mass_msun'].apply(mass_to_luminosity) 

# Calculate HZ boundaries and add them to the dataframe 
data[['HZ_inner_AU_s', 'HZ_outer_AU_s']] = data['Luminosity_Lsun'].apply( lambda luminosity: calculate_habitable_zone(luminosity) ).apply(pd.Series)

# Display the updated dataset to the user in a Jupyter-friendly format 
print(data)

3.2 Version 2: More complex

In [6]:
# Function to convert mass to luminosity 
def mass_to_luminosity(star_mass_msun):
    return star_mass_msun  ** 3.6

# Function to calculate habitable zone boundaries 
def calculate_habitable_zone(Luminosity_Lsun):
    inner_boundary = np.sqrt(Luminosity_Lsun / 1.1)
    outer_boundary = np.sqrt(Luminosity_Lsun / 0.53)
    return inner_boundary, outer_boundary 

# Convert mass to luminosity 
data['Luminosity_Lsun'] = data['star_mass_msun'].apply(mass_to_luminosity) 



# inner flux 
def s_inner(Teff):
    s_inner = 1.296 - 2.139e-4 * Teff + 4.19e-8 * (Teff)**2

    return s_inner

data['Flux_inner_HZ'] = data['star_teff_K'].apply(s_inner) 

# outer flux
def s_outer(Teff):
    s_outer = 0.234 - 1.319e-5 * Teff + 6.19e-10 * (Teff)**2

    return s_outer

data['Flux_outer_HZ'] = data['star_teff_K'].apply(s_outer) 

# inner boundary
def r_inner(Luminosity_Lsun, Flux_inner_HZ):
    r_inner = np.sqrt(Luminosity_Lsun / Flux_inner_HZ)

    return r_inner

data['HZ_inner_AU_2'] = data.apply(lambda row: r_inner(row['Luminosity_Lsun'], row['Flux_inner_HZ']), axis = 1)


# outer boundary
def r_outer(Luminosity_Lsun, Flux_outer_HZ):
    r_outer = np.sqrt(Luminosity_Lsun / Flux_outer_HZ)

    return r_outer

data['HZ_outer_AU_2'] = data.apply(lambda row: r_outer(row['Luminosity_Lsun'], row['Flux_outer_HZ']), axis = 1)

# print(data)

4. Calculate middle of HZ

In [8]:
def HZ_middle_ideal(HZ_inner, HZ_outer):
    HZ_middle = (HZ_inner + HZ_outer) / 2

    return HZ_middle_ideal

# data['HZ_middle_AU_s'] = data.apply(lambda row: HZ_middle(row['HZ_inner_AU_s'], row['HZ_outer_AU_s']), axis = 1)
data['HZ_middle_ideal_AU'] = data.apply(lambda row: HZ_middle_ideal(row['HZ_inner_AU_2'], row['HZ_outer_AU_2']), axis = 1)


5. Radiation Intensity
- Calculate distance from planet to star in m

In [9]:
def pc_to_m(star_distance_pc):
    star_distance_m = star_distance_pc * 3.086e+16

    return star_distance_m

data['star_distance_m'] = data['star_distance_pc'].apply(pc_to_m) 

# print(data)


In [10]:
# radiation = L / 4pi(r)^2 W/m^2
def radiation_I(luminosity, distance_m):
    radiation_I = luminosity / (4*np.pi*(distance_m**2))

    return radiation_I


data['radiation_I41'] = data.apply(lambda row: radiation_I(row['Luminosity_Lsun'], row['star_distance_m']), axis = 1)

In [ ]:
print(data)



5. Calculate pre-main sequence time and main-sequence lifetime for given stars. ideal = middle main sequence, non-ideal = pre or post-main sequence

5.1 Calculate pre-main sequence phase (tPMS)

In [11]:
def pre_seq_GY(star_mass_msun):
    pre_seq_length = ( 10e7*(star_mass_msun**-2.5) ) / 10e9

    return pre_seq_length

data['pre_seq_GY'] = data['star_mass_msun'].apply(pre_seq_GY) 
# print(data)

5.2 Calculate main-sequence lifetime (tMS)

In [12]:
def main_seq_GY(star_mass_msun):
    main_seq_length = 10*(1/(star_mass_msun**2.5))

    return main_seq_length

data['main_seq_GY'] = data['star_mass_msun'].apply(main_seq_GY) 
# print(data)

5.3 Calculate end of main sequence phase (transition into post-main sequence)

In [13]:
data['main_seq_end_GY'] = data['pre_seq_GY'] + data['main_seq_GY']

# print(data)

5.4 Calculate mid-main sequence time

In [14]:
def mid_main_seq_GY(main_seq_end_GY, pre_seq_GY):
    mid_main_seq = (main_seq_end_GY - pre_seq_GY) / 2

    return mid_main_seq

data['mid_main_seq_GY'] = data.apply(lambda row: mid_main_seq_GY(row['main_seq_end_GY'], row['pre_seq_GY']), axis = 1)
# print(data)

6. Calculate ideal and non-ideal values ??

- planet mass M(earth) == Earth? between 0.5 and 5 earth masses as can maintain atmosphere and possibly have H2O (aq)
- planet radius R(earth) == Earth? between 0.8 and 1.5 earth radii as above might have thick gas envelopes and be more like mini-neptures/gas giant
- planet temperature (K) == Earth? between 0 and 100'c where water can remain liquid as standard pressure
- planet density == Earth? between 5 and 5.5 g/cm3
- radiation intensity (W/m^2) == Earth ? limits at HZ?
- HZ measurement == normal distribution, y = middle of HZ; non ideal = | 2sigma | where 1.5 sigma == boundary 
- star age ?? Gy maybe calculate lifetime based on Tms = 10gy * (1/(Mstar^2.5))  where Tms is main sequence lifetime. Pre-main sequence phase tpms = 10^7 * M^-2.5
- interactions??

6.1 Mass ideal and non-ideal values

In [ ]:
# ?????

6.2 Radius ideal and non-ideal values

In [ ]:
# ?????
print(data)

6.3 Temperature ideal and non-ideal values

In [ ]:
# calculate temperature in C

def K_to_C(temp_calculated_K):

    temp_calculated_C = temp_calculated_K - 273

    return temp_calculated_C
data['temp_calculated_C'] = data['temp_calculated_K'].apply(K_to_C) 


## ideal = 20
## non ideal: 0 and 100

6.4 Density ideal and non-ideal values

In [ ]:
# not sure if this is relevant since densities varysignigicantly 

6.5 Radiation intensity

In [ ]:
# ideal = intensity at UB?
# non-ideal = intensity at LB?

# print(data)

In [ ]:
# radiation = L / 4pi(r)^2 W/m^2
# translate to m
def radiation_I(luminosity, distance_m):
    radiation_I = luminosity / (4*np.pi*(distance_m**2))

    return radiation_I


data['radiation_I41'] = data.apply(lambda row: radiation_I(row['Luminosity_Lsun'], row['star_distance_m']), axis = 1)

6.6 HZ normal distribution; ideal and non-ideal values

In [15]:
def HZ_normal_curve(data):

    lower_bound_values = []
    upper_bound_values = []

    for index, row in data.iterrows():
        midpoint = row["HZ_middle_AU_2"]
        outer_boundary = row["HZ_outer_AU_2"]

        # Calculate bounds for values exactly 2 std deviations away from the mean
        lower_bound = midpoint - 2 * ((outer_boundary - midpoint) / 1.5)
        upper_bound = midpoint + 2 * ((outer_boundary - midpoint) / 1.5)
        
        # Append values to lists
        lower_bound_values.append(lower_bound)
        upper_bound_values.append(upper_bound)
    
    # Add new columns to DataFrame
    data['HZ_LB_nonideal'] = lower_bound_values
    data['HZ_UB_nonideal'] = upper_bound_values
    
    return data

# Apply the function to your dataset
data = HZ_normal_curve(data)

# Display the updated dataset with specific values at lower and upper bounds recorded
# print(data)




6.6 Star age ideal and non-ideal values

In [ ]:
print(data)

In [27]:
# ideal = mid_main_seq_GY  
# non ideal: > main_seq_end_GY  OR < pre_seq_GY  

7. Normalise decision matrix ?

8. Weighing
Weighing can be determined in 1 of 2 ways:
- use a correlation regression as done in Cobb Douglas
- use difference from "ideal" Earth on a normal distribution to determine the importance of outliers

9. Seperation measures - distance to ideal and non-ideal
- likely to include creation of a completely new dataset for comparison of EACH exoplanet

10. Relative closeness to ideal

11. Comparison to Earth etc
Would be good to add a visual aspect:
- maybe an image with 'sliders' showing where points are from ideal?
-  